In [1]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import glob
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.utils import save_image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [2]:
# Root directory for dataset
data_root = "/datasets/home/32/232/tdobhal/Project/6_train/images/"

# Number of images in the directory
num_images = 23418

# Batch size during training
batch_size = 16

# Spatial size of training images. All images will be resized to this size using a transformer.
image_size = 256

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 100

# Learning rate for optimizers
lr = 0.00005

# Alpha hyperparam for RMS optimizers
alpha = 0.9

# Number of GPUs available. Use 0 for CPU mode.
ngpu = torch.cuda.device_count()

# Device to run on
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
class DataLoader:
    def __init__(self):
        '''
        Parameters:
        
        '''
        self.device = device
        self.data_path = data_root
        self.image_size = image_size
        self.batch_size = batch_size
        self.train_names = glob.glob(self.data_path + 'real_A/*')
        self.names = [self.train_names[i].split('/')[-1] for i in range(len(self.train_names))]
        self.data_transforms = torchvision.transforms.Compose([
                torchvision.transforms.Resize(image_size),
                torchvision.transforms.CenterCrop(image_size),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
    
    def image_loader(self, image_name):
        """load image, returns cuda tensor"""
        image = Image.open(image_name)
        image = self.data_transforms(image).float()
        image = torch.autograd.Variable(image, requires_grad=True)
        image = image.unsqueeze(0)  # this is for VGG, may not be needed for ResNet
        return image[0].to(self.device)  # assumes that you're using GPU

    def show(self, img):
        npimg = img.cpu().detach().numpy()
        npimg = np.transpose(npimg, (1,2,0))
        if npimg.shape[2] == 3:
            plt.imshow(npimg)
        else:
            plt.imshow(npimg[:,:,0], cmap='gray')
            
    def imshow(self, img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.cpu().detach().numpy()
        plt.figure(figsize = (10,2))
        plt.imshow(np.transpose(npimg, (1, 2, 0)), aspect='auto')

    def data_generator(self):
        root = self.data_path
        batch_size = self.batch_size
        
        images_dir = root + 'real_A/'
        labels_dir = root + 'fake_B/'

        while True:
            x, y = [], []
            idx = np.random.choice(self.names, batch_size)
            for i in range(idx.shape[0]):
                x.append(self.image_loader(images_dir + idx[i]))
                y.append(self.image_loader(labels_dir + idx[i]))
            yield torch.stack(x), torch.stack(y)

In [4]:
data = DataLoader()

In [5]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.01)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.01)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        
        # Convolution layers
        
        # input is (nc) x 256 x 256
        self.conv1 = nn.Conv2d(nc, ngf, 4, 2, 1, bias=True)
        self.lr1 = nn.LeakyReLU(inplace=True)
        # state size. (ngf) x 128 x 128
        self.conv2 = nn.Conv2d(ngf, ngf*2, 4, 2, 1, bias=True)
        self.bn2 = nn.BatchNorm2d(ngf*2)
        self.lr2 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*2) x 64 x 64
        self.conv3 = nn.Conv2d(ngf*2, ngf*4, 4, 2, 1, bias=True)
        self.bn3 = nn.BatchNorm2d(ngf*4)
        self.lr3 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*4) x 32 x 32
        self.conv4 = nn.Conv2d(ngf*4, ngf*8, 4, 2, 1, bias=True)
        self.bn4 = nn.BatchNorm2d(ngf*8)
        self.lr4 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 16 x 16
        self.conv5 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn5 = nn.BatchNorm2d(ngf*8)
        self.lr5 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 8 x 8
        self.conv6 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn6 = nn.BatchNorm2d(ngf*8)
        self.lr6 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 4 x 4
        self.conv7 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn7 = nn.BatchNorm2d(ngf*8)
        self.lr7 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 2 x 2
        self.conv8 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn8 = nn.BatchNorm2d(ngf*8)
        self.r8 = nn.ReLU(inplace=True)
        
        # Transpose Convolutional Layers
        
        # input is (ngf*8) x 1 x 1
        self.tr_conv1 = nn.ConvTranspose2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn1 = nn.BatchNorm2d(ngf*8)
        self.tr_r1 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 2 x 2
        self.tr_conv2 = nn.ConvTranspose2d((ngf*8)*2, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn2 = nn.BatchNorm2d(ngf*8)
        self.tr_r2 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 4 x 4
        self.tr_conv3 = nn.ConvTranspose2d((ngf*8)*2, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn3 = nn.BatchNorm2d(ngf*8)
        self.tr_r3 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 8 x 8
        self.tr_conv4 = nn.ConvTranspose2d((ngf*8)*2, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn4 = nn.BatchNorm2d(ngf*8)
        self.tr_r4 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 16 x 16
        self.tr_conv5 = nn.ConvTranspose2d((ngf*8)*2, ngf*4, 4, 2, 1, bias=True)
        self.tr_bn5 = nn.BatchNorm2d(ngf*4)
        self.tr_r5 = nn.ReLU(inplace=True)
        # state size. (ngf*4)*2 x 32 x 32
        self.tr_conv6 = nn.ConvTranspose2d((ngf*4)*2, ngf*2, 4, 2, 1, bias=True)
        self.tr_bn6 = nn.BatchNorm2d(ngf*2)
        self.tr_r6 = nn.ReLU(inplace=True)
        # state size. (ngf*2)*2 x 64 x 64
        self.tr_conv7 = nn.ConvTranspose2d((ngf*2)*2, ngf, 4, 2, 1, bias=True)
        self.tr_bn7 = nn.BatchNorm2d(ngf)
        self.tr_r7 = nn.ReLU(inplace=True)
        # state size. (ngf)*2 x 128 x 128
        self.tr_conv8 = nn.ConvTranspose2d((ngf)*2, nc, 4, 2, 1, bias=True)
        self.out = nn.Tanh()
        # state size. (nc) x 256 x 256
    
    def forward(self, x):
        c1 = self.conv1(x)
        c2 = self.bn2(self.conv2(self.lr1(c1)))
        c3 = self.bn3(self.conv3(self.lr2(c2)))
        c4 = self.bn4(self.conv4(self.lr3(c3)))
        c5 = self.bn5(self.conv5(self.lr4(c4)))
        c6 = self.bn6(self.conv6(self.lr5(c5)))
        c7 = self.bn7(self.conv7(self.lr6(c6)))
        c8 = self.bn8(self.conv8(self.lr7(c7)))
        
        t1 = self.tr_bn1(self.tr_conv1(self.r8(c8)))
        t1 = torch.cat((t1, c7), dim=1)
        t2 = self.tr_bn2(self.tr_conv2(self.tr_r1(t1)))
        t2 = torch.cat((t2, c6), dim=1)
        t3 = self.tr_bn3(self.tr_conv3(self.tr_r2(t2)))
        t3 = torch.cat((t3, c5), dim=1)
        t4 = self.tr_bn4(self.tr_conv4(self.tr_r3(t3)))
        t4 = torch.cat((t4, c4), dim=1)
        t5 = self.tr_bn5(self.tr_conv5(self.tr_r4(t4)))
        t5 = torch.cat((t5, c3), dim=1)
        t6 = self.tr_bn6(self.tr_conv6(self.tr_r5(t5)))
        t6 = torch.cat((t6, c2), dim=1)
        t7 = self.tr_bn7(self.tr_conv7(self.tr_r6(t6)))
        t7 = torch.cat((t7, c1), dim=1)
        t8 = self.tr_conv8(self.tr_r7(t7))
        t8 = self.out(t8)
        return t8
    

In [7]:
gen_a = Generator().to(device)
gen_b = Generator().to(device)

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            # input is (nc) x 256 x 256
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=True),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 128 x 128
            nn.MaxPool2d((2, 2)), 
            
            # state size. (ndf) x 64 x 64
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. 32 x 32
            nn.MaxPool2d((2, 2)),
    
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, ndf * 8, 4, 2, 1, bias=True),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            )
        
        self.flat = nn.Linear(ndf * 8 * 2 * 2, 1)
        self.out = nn.Sigmoid()
        
    def forward(self, x):
        x = self.net(x)
        x = x.view(x.size()[0], -1)
        x = self.flat(x)
        x = self.out(x)
        return x

In [9]:
dis_a = Discriminator().to(device)
dis_b = Discriminator().to(device)

In [10]:
gen_a = nn.DataParallel(gen_a, list(range(ngpu)))
dis_a = nn.DataParallel(dis_a, list(range(ngpu)))
gen_b = nn.DataParallel(gen_b, list(range(ngpu)))
dis_b = nn.DataParallel(dis_b, list(range(ngpu)))

gen_a.apply(weights_init_normal)
dis_a.apply(weights_init_normal)
gen_b.apply(weights_init_normal)
dis_b.apply(weights_init_normal)

DataParallel(
  (module): Discriminator(
    (net): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2, inplace)
      (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.2, inplace)
      (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): LeakyReLU(negative_slope=0.2, inplace)
      (10): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (11): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [11]:
criterion = torch.nn.BCELoss()

optim_gen_a = torch.optim.RMSprop(gen_a.parameters(), lr, alpha)
optim_gen_b = torch.optim.RMSprop(gen_b.parameters(), lr, alpha)
optim_dis_a = torch.optim.RMSprop(dis_a.parameters(), lr, alpha)
optim_dis_b = torch.optim.RMSprop(dis_b.parameters(), lr, alpha)

### Train Loop

In [ ]:
sample_interval = 25
checkpoint_interval = 1

for epoch in range(num_epochs):
    for i in range(num_images // batch_size):
        x, y = next(data.data_generator())
        real_a = Variable(x).to(device)
        real_b = Variable(y).to(device)     
        valid = Variable(torch.ones((real_a.size(0), 1)), requires_grad=False).to(device)
        fake = Variable(torch.zeros((real_a.size(0), 1)), requires_grad=False).to(device)
        
        # Training Discriminator A with real_A batch
        optim_dis_a.zero_grad();
        pred_real_dis_a = dis_a(real_a).view(-1, 1)
        err_real_dis_a = criterion(pred_real_dis_a, valid)
        err_real_dis_a.backward()
        
        # Training Discriminator B with real_B batch
        optim_dis_b.zero_grad();
        pred_real_dis_b = dis_b(real_b).view(-1, 1)
        err_real_dis_b = criterion(pred_real_dis_b, valid)
        err_real_dis_b.backward()
        
        # Training Discriminator B with fake_B batch of Generator A
        fake_b = gen_a(real_a)
        pred_fake_dis_b = dis_b(fake_b.detach()).view(-1, 1)
        err_fake_dis_b = criterion(pred_fake_dis_b, fake)
        err_fake_dis_b.backward()
        
        # Training Discriminator A with fake_A batch of Generator B
        fake_a = gen_b(real_b)
        pred_fake_dis_a = dis_a(fake_a.detach()).view(-1, 1)
        err_fake_dis_a = criterion(pred_fake_dis_a, fake)
        err_fake_dis_a.backward()
        
        # Update params of Discriminator A and B
        err_dis_a = err_real_dis_a + err_fake_dis_a
        optim_dis_a.step()
        err_dis_b = err_real_dis_b + err_fake_dis_b
        optim_dis_b.step()
        
        # Train and update Generator A based on Discriminator B's prediction
        optim_gen_a.zero_grad()
        pred_out_dis_b = dis_b(fake_b).view(-1, 1)
        err_gen_a = criterion(pred_out_dis_b, valid)
        err_gen_a.backward()
        optim_gen_a.step()
        
        # Train and update Generator B based on Discriminator A's prediction
        optim_gen_b.zero_grad()
        pred_out_dis_a = dis_a(fake_a).view(-1, 1)
        err_gen_b = criterion(pred_out_dis_a, valid)
        err_gen_b.backward()
        optim_gen_b.step()
        
        # Print statistics and save checkpoints
        print("\r[Epoch %d/%d] [Batch %d/%d] [D_A loss: %f] [D_B loss: %f] [G_A loss: %f, G_B loss: %f]" %
                                                        (epoch, num_epochs,
                                                        i, num_images//batch_size,
                                                        err_dis_a.item(), err_dis_b.item(), 
                                                        err_gen_a.item(), err_gen_b.item()))

        if i % sample_interval == 0:
            img_sample = torch.cat((real_a.data, fake_a.data, real_b.data, fake_b.data), -2)
            save_image(img_sample, 'saved_images/%s.png' % (epoch + i), nrow=5, normalize=True)


    if checkpoint_interval != -1 and epoch % checkpoint_interval == 0:
        torch.save(gen_a.state_dict(), 'saved_models/generator_a_%d.pth' % (epoch))
        torch.save(gen_b.state_dict(), 'saved_models/generator_b_%d.pth' % (epoch))
        torch.save(dis_a.state_dict(), 'saved_models/discriminator_a_%d.pth' % (epoch))
        torch.save(dis_b.state_dict(), 'saved_models/discriminator_b_%d.pth' % (epoch))

[Epoch 0/100] [Batch 0/1463] [D_A loss: 1.346673] [D_B loss: 1.391420] [G_A loss: 5.447925, G_B loss: 5.793624]
[Epoch 0/100] [Batch 1/1463] [D_A loss: 0.242034] [D_B loss: 0.355362] [G_A loss: 0.935152, G_B loss: 1.980854]
[Epoch 0/100] [Batch 2/1463] [D_A loss: 1.637061] [D_B loss: 2.425555] [G_A loss: 5.972492, G_B loss: 6.427376]
[Epoch 0/100] [Batch 3/1463] [D_A loss: 0.863290] [D_B loss: 1.159959] [G_A loss: 2.181315, G_B loss: 3.224128]
[Epoch 0/100] [Batch 4/1463] [D_A loss: 0.321887] [D_B loss: 0.416616] [G_A loss: 3.973685, G_B loss: 4.434943]
[Epoch 0/100] [Batch 5/1463] [D_A loss: 0.159926] [D_B loss: 0.370327] [G_A loss: 2.795234, G_B loss: 3.434728]
[Epoch 0/100] [Batch 6/1463] [D_A loss: 0.306757] [D_B loss: 0.726021] [G_A loss: 5.074713, G_B loss: 4.986984]
[Epoch 0/100] [Batch 7/1463] [D_A loss: 0.315539] [D_B loss: 0.826276] [G_A loss: 0.881441, G_B loss: 2.081090]
[Epoch 0/100] [Batch 8/1463] [D_A loss: 0.491640] [D_B loss: 1.199163] [G_A loss: 4.870562, G_B loss: 5.

[Epoch 0/100] [Batch 73/1463] [D_A loss: 0.976657] [D_B loss: 1.838124] [G_A loss: 3.362282, G_B loss: 0.525094]
[Epoch 0/100] [Batch 74/1463] [D_A loss: 1.616441] [D_B loss: 1.533950] [G_A loss: 0.264423, G_B loss: 4.584720]
[Epoch 0/100] [Batch 75/1463] [D_A loss: 2.186547] [D_B loss: 1.633555] [G_A loss: 2.590693, G_B loss: 0.398574]
[Epoch 0/100] [Batch 76/1463] [D_A loss: 1.072684] [D_B loss: 1.277294] [G_A loss: 0.543051, G_B loss: 2.677326]
[Epoch 0/100] [Batch 77/1463] [D_A loss: 0.821480] [D_B loss: 1.206126] [G_A loss: 2.388671, G_B loss: 0.920285]
[Epoch 0/100] [Batch 78/1463] [D_A loss: 0.731072] [D_B loss: 1.010149] [G_A loss: 0.661382, G_B loss: 3.255643]
[Epoch 0/100] [Batch 79/1463] [D_A loss: 1.444063] [D_B loss: 1.456120] [G_A loss: 2.649773, G_B loss: 0.189719]
[Epoch 0/100] [Batch 80/1463] [D_A loss: 1.968631] [D_B loss: 1.135415] [G_A loss: 0.599193, G_B loss: 3.246586]
[Epoch 0/100] [Batch 81/1463] [D_A loss: 1.224807] [D_B loss: 1.165106] [G_A loss: 2.868992, G_B

[Epoch 0/100] [Batch 146/1463] [D_A loss: 1.491847] [D_B loss: 1.443757] [G_A loss: 0.895341, G_B loss: 0.513971]
[Epoch 0/100] [Batch 147/1463] [D_A loss: 1.467202] [D_B loss: 1.506407] [G_A loss: 0.742209, G_B loss: 1.255419]
[Epoch 0/100] [Batch 148/1463] [D_A loss: 1.472593] [D_B loss: 1.484068] [G_A loss: 0.943037, G_B loss: 0.598627]
[Epoch 0/100] [Batch 149/1463] [D_A loss: 1.488975] [D_B loss: 1.549402] [G_A loss: 0.732039, G_B loss: 0.889892]
[Epoch 0/100] [Batch 150/1463] [D_A loss: 1.406123] [D_B loss: 1.450393] [G_A loss: 0.895989, G_B loss: 0.845586]
[Epoch 0/100] [Batch 151/1463] [D_A loss: 1.452981] [D_B loss: 1.441748] [G_A loss: 0.812508, G_B loss: 0.802523]
[Epoch 0/100] [Batch 152/1463] [D_A loss: 1.421034] [D_B loss: 1.440423] [G_A loss: 0.939108, G_B loss: 1.135216]
[Epoch 0/100] [Batch 153/1463] [D_A loss: 1.469645] [D_B loss: 1.471168] [G_A loss: 0.751364, G_B loss: 0.473906]
[Epoch 0/100] [Batch 154/1463] [D_A loss: 1.492146] [D_B loss: 1.416489] [G_A loss: 1.03

[Epoch 0/100] [Batch 218/1463] [D_A loss: 1.434362] [D_B loss: 1.439059] [G_A loss: 0.856337, G_B loss: 0.625369]
[Epoch 0/100] [Batch 219/1463] [D_A loss: 1.469218] [D_B loss: 1.444087] [G_A loss: 0.727650, G_B loss: 0.915859]
[Epoch 0/100] [Batch 220/1463] [D_A loss: 1.525594] [D_B loss: 1.441754] [G_A loss: 0.833899, G_B loss: 0.841225]
[Epoch 0/100] [Batch 221/1463] [D_A loss: 1.396624] [D_B loss: 1.423615] [G_A loss: 0.745115, G_B loss: 0.763644]
[Epoch 0/100] [Batch 222/1463] [D_A loss: 1.398333] [D_B loss: 1.425818] [G_A loss: 0.757420, G_B loss: 0.732383]
[Epoch 0/100] [Batch 223/1463] [D_A loss: 1.427708] [D_B loss: 1.415908] [G_A loss: 0.848170, G_B loss: 0.745645]
[Epoch 0/100] [Batch 224/1463] [D_A loss: 1.413277] [D_B loss: 1.410570] [G_A loss: 0.699876, G_B loss: 0.799231]
[Epoch 0/100] [Batch 225/1463] [D_A loss: 1.425462] [D_B loss: 1.452962] [G_A loss: 0.939777, G_B loss: 0.680230]
[Epoch 0/100] [Batch 226/1463] [D_A loss: 1.408016] [D_B loss: 1.434764] [G_A loss: 0.72

[Epoch 0/100] [Batch 290/1463] [D_A loss: 1.406568] [D_B loss: 1.408033] [G_A loss: 0.745443, G_B loss: 0.693636]
[Epoch 0/100] [Batch 291/1463] [D_A loss: 1.405096] [D_B loss: 1.411905] [G_A loss: 0.739841, G_B loss: 0.738772]
[Epoch 0/100] [Batch 292/1463] [D_A loss: 1.404773] [D_B loss: 1.405772] [G_A loss: 0.730675, G_B loss: 0.684181]
[Epoch 0/100] [Batch 293/1463] [D_A loss: 1.398589] [D_B loss: 1.405707] [G_A loss: 0.695954, G_B loss: 0.824222]
[Epoch 0/100] [Batch 294/1463] [D_A loss: 1.407291] [D_B loss: 1.411144] [G_A loss: 0.754372, G_B loss: 0.655458]
[Epoch 0/100] [Batch 295/1463] [D_A loss: 1.420009] [D_B loss: 1.399875] [G_A loss: 0.731598, G_B loss: 0.835952]
[Epoch 0/100] [Batch 296/1463] [D_A loss: 1.417316] [D_B loss: 1.405336] [G_A loss: 0.707501, G_B loss: 0.670478]
[Epoch 0/100] [Batch 297/1463] [D_A loss: 1.423547] [D_B loss: 1.423031] [G_A loss: 0.808645, G_B loss: 0.770859]
[Epoch 0/100] [Batch 298/1463] [D_A loss: 1.399212] [D_B loss: 1.404583] [G_A loss: 0.72